# 16 Implement DQN (Deep Q-Networks)

DQN (Deep Q-Network) Project: Cart and Pole 

The original tutorial for which this project is based is located at the link below. 

I have made several modifications throughout.

https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html

# 16.1 Import Libraries

In [15]:
# Import Libraries
print('import Library..')
%matplotlib inline
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

## https://github.com/pytorch/vision
## conda install torchvision -c pytorch

# https://stackoverflow.com/questions/59621736/despite-installing-the-torch-vision-pytorch-library-i-am-getting-an-error-sayin
# conda install pytorch torchvision cpuonly -c pytorch
import torchvision.transforms as T

import Library..


# 16.2 Set up display

In [12]:
# Import Libraries
print('Import Libraries...')
print()
%matplotlib inline
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

## https://github.com/pytorch/vision
## conda install torchvision -c pytorch

# https://stackoverflow.com/questions/59621736/despite-installing-the-torch-vision-pytorch-library-i-am-getting-an-error-sayin
# conda install pytorch torchvision cpuonly -c pytorch
import torchvision.transforms as T

# Setup Display for plotting images
print('Setup Dsiplay...')
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython: from IPython import display

Import Libraries...

Setup Dsiplay...


# 16.3 Deep Q-Network

In [3]:
# Import Libraries
print('Import Libraries...')
print()
%matplotlib inline
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

## https://github.com/pytorch/vision
## conda install torchvision -c pytorch

# https://stackoverflow.com/questions/59621736/despite-installing-the-torch-vision-pytorch-library-i-am-getting-an-error-sayin
# conda install pytorch torchvision cpuonly -c pytorch
import torchvision.transforms as T

# Setup Display for plotting images
print('Setup Dsiplay...')
print()
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython: from IPython import display
    
# Define Class DQN and inherit torch.nn.Module 
print('Define class DQN...')
class DQN(nn.Module):
    def __init__(self, img_height, img_width):
        super().__init__()
         
        self.fc1 = nn.Linear(in_features=img_height*img_width*3, out_features=24)   
        self.fc2 = nn.Linear(in_features=24, out_features=32)
        self.out = nn.Linear(in_features=32, out_features=2)            

    def forward(self, t):
        t = t.flatten(start_dim=1)
        t = F.relu(self.fc1(t))
        t = F.relu(self.fc2(t))
        t = self.out(t)
        return t

Import Libraries...

Setup Dsiplay...

Define class DQN...


# 16.4 Experience Class

In [13]:
# Import Libraries
print('Import Library...')
print()
%matplotlib inline
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

## https://github.com/pytorch/vision
## conda install torchvision -c pytorch

# https://stackoverflow.com/questions/59621736/despite-installing-the-torch-vision-pytorch-library-i-am-getting-an-error-sayin
# conda install pytorch torchvision cpuonly -c pytorch
import torchvision.transforms as T

# Setup Display for plotting images
print('Setup Dsiplay...')
print()
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython: from IPython import display
    
# Define Class DQN and inherit torch.nn.Module 
print('Define class DQN...')
print()
class DQN(nn.Module):
    def __init__(self, img_height, img_width):
        super().__init__()
         
        self.fc1 = nn.Linear(in_features=img_height*img_width*3, out_features=24)   
        self.fc2 = nn.Linear(in_features=24, out_features=32)
        self.out = nn.Linear(in_features=32, out_features=2)            

    def forward(self, t):
        t = t.flatten(start_dim=1)
        t = F.relu(self.fc1(t))
        t = F.relu(self.fc2(t))
        t = self.out(t)
        return t
# Use factory function 'namedtuple' to craete the class 'Experience'.
#
# How to use factory fuction 'namedtuple' to create the class?
# https://realpython.com/python-namedtuple/
#
print('Create Class Experience for replay memory...')
print()
Experience = namedtuple(
    'Experience',
    ('state', 'action', 'next_state', 'reward')
)

# Create expeirnece instance
print('creat instance e...')
print(e)
e = Experience(2,3,1,4)

Import Library...

Setup Dsiplay...

Define class DQN...

Create Class Experience for replay memory...

creat instance e...
Experience(state=2, action=3, next_state=1, reward=4)


# 16.5 Replay Memory

In [12]:
# Import Libraries
print('Import Library...')
print()
%matplotlib inline
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

## https://github.com/pytorch/vision
## conda install torchvision -c pytorch

# https://stackoverflow.com/questions/59621736/despite-installing-the-torch-vision-pytorch-library-i-am-getting-an-error-sayin
# conda install pytorch torchvision cpuonly -c pytorch
import torchvision.transforms as T

# Setup Display for plotting images
print('Setup Dsiplay...')
print()
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython: from IPython import display
    
# Define Class DQN and inherit torch.nn.Module 
print('Define class DQN...')
print()
class DQN(nn.Module):
    def __init__(self, img_height, img_width):
        super().__init__()
         
        self.fc1 = nn.Linear(in_features=img_height*img_width*3, out_features=24)   
        self.fc2 = nn.Linear(in_features=24, out_features=32)
        self.out = nn.Linear(in_features=32, out_features=2)            

    def forward(self, t):
        t = t.flatten(start_dim=1)
        t = F.relu(self.fc1(t))
        t = F.relu(self.fc2(t))
        t = self.out(t)
        return t

# Use factory function 'namedtuple' to craete the class 'Experience'.
#
# How to use factory fuction 'namedtuple' to create the class?
# https://realpython.com/python-namedtuple/
#
print('Create Class Experience for replay memory...')
print()
Experience = namedtuple(
    'Experience',
    ('state', 'action', 'next_state', 'reward')
)

# Create expeirnece instance
print('creat instance e ...')
print(e)
print()
e = Experience(2,3,1,4)

print('Define ReplayMemory()...')
class ReplayMemory():
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.push_count = 0
        
    def push(self, experience):
        if len(self.memory) < self.capacity:
            self.memory.append(experience)
        else:
            self.memory[self.push_count % self.capacity] = experience
        self.push_count += 1

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def can_provide_sample(self, batch_size):
        return len(self.memory) >= batch_size

Import Library...

Setup Dsiplay...

Define class DQN...

Create Class Experience for replay memory...

creat instance e ...
Experience(state=2, action=3, next_state=1, reward=4)

Define ReplayMemory()...


# 16.6 Epsilon Greedy Strategy

In [14]:
# Import Libraries
print('Import Library...')
print()
%matplotlib inline
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

## https://github.com/pytorch/vision
## conda install torchvision -c pytorch

# https://stackoverflow.com/questions/59621736/despite-installing-the-torch-vision-pytorch-library-i-am-getting-an-error-sayin
# conda install pytorch torchvision cpuonly -c pytorch
import torchvision.transforms as T

# Setup Display for plotting images
print('Setup Dsiplay...')
print()
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython: from IPython import display
    
# Define Class DQN and inherit torch.nn.Module 
print('Define class DQN...')
print()
class DQN(nn.Module):
    def __init__(self, img_height, img_width):
        super().__init__()
         
        self.fc1 = nn.Linear(in_features=img_height*img_width*3, out_features=24)   
        self.fc2 = nn.Linear(in_features=24, out_features=32)
        self.out = nn.Linear(in_features=32, out_features=2)            

    def forward(self, t):
        t = t.flatten(start_dim=1)
        t = F.relu(self.fc1(t))
        t = F.relu(self.fc2(t))
        t = self.out(t)
        return t

# Use factory function 'namedtuple' to craete the class 'Experience'.
#
# How to use factory fuction 'namedtuple' to create the class?
# https://realpython.com/python-namedtuple/
#
print('Create Class Experience for replay memory...')
print()
Experience = namedtuple(
    'Experience',
    ('state', 'action', 'next_state', 'reward')
)

# Create expeirnece instance
print('creat instance e ...')
print(e)
print()
e = Experience(2,3,1,4)

print('Define ReplayMemory()...')
print()
class ReplayMemory():
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.push_count = 0
        
    def push(self, experience):
        if len(self.memory) < self.capacity:
            self.memory.append(experience)
        else:
            self.memory[self.push_count % self.capacity] = experience
        self.push_count += 1

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def can_provide_sample(self, batch_size):
        return len(self.memory) >= batch_size
print('define class EpsilonGreedyStrategy()...')
class EpsilonGreedyStrategy():
    def __init__(self, start, end, decay):
        self.start = start
        self.end = end
        self.decay = decay
    
    def get_exploration_rate(self, current_step):
        return self.end + (self.start - self.end) * \
            math.exp(-1. * current_step * self.decay)

Import Library...

Setup Dsiplay...

Define class DQN...

Create Class Experience for replay memory...

creat instance e ...
Experience(state=2, action=3, next_state=1, reward=4)

Define ReplayMemory()...

define class EpsilonGreedyStrategy()...


# 16.7 Reinforcement Learning Agent

In [16]:
# Import Libraries
print('Import Library...')
print()
%matplotlib inline
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

## https://github.com/pytorch/vision
## conda install torchvision -c pytorch

# https://stackoverflow.com/questions/59621736/despite-installing-the-torch-vision-pytorch-library-i-am-getting-an-error-sayin
# conda install pytorch torchvision cpuonly -c pytorch
import torchvision.transforms as T

# Setup Display for plotting images
print('Setup Dsiplay...')
print()
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython: from IPython import display
    
# Define Class DQN and inherit torch.nn.Module 
print('Define class DQN...')
print()
class DQN(nn.Module):
    def __init__(self, img_height, img_width):
        super().__init__()
         
        self.fc1 = nn.Linear(in_features=img_height*img_width*3, out_features=24)   
        self.fc2 = nn.Linear(in_features=24, out_features=32)
        self.out = nn.Linear(in_features=32, out_features=2)            

    def forward(self, t):
        t = t.flatten(start_dim=1)
        t = F.relu(self.fc1(t))
        t = F.relu(self.fc2(t))
        t = self.out(t)
        return t

# Use factory function 'namedtuple' to craete the class 'Experience'.
#
# How to use factory fuction 'namedtuple' to create the class?
# https://realpython.com/python-namedtuple/
#
print('Create Class Experience for replay memory...')
print()
Experience = namedtuple(
    'Experience',
    ('state', 'action', 'next_state', 'reward')
)

# Create expeirnece instance
print('creat instance e ...')
print(e)
print()
e = Experience(2,3,1,4)

print('Define ReplayMemory()...')
print()
class ReplayMemory():
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.push_count = 0
        
    def push(self, experience):
        if len(self.memory) < self.capacity:
            self.memory.append(experience)
        else:
            self.memory[self.push_count % self.capacity] = experience
        self.push_count += 1

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def can_provide_sample(self, batch_size):
        return len(self.memory) >= batch_size

print('define class EpsilonGreedyStrategy()...')
print()
class EpsilonGreedyStrategy():
    def __init__(self, start, end, decay):
        self.start = start
        self.end = end
        self.decay = decay
    
    def get_exploration_rate(self, current_step):
        return self.end + (self.start - self.end) * \
            math.exp(-1. * current_step * self.decay)

    print('Define class Agent()...')
class Agent():
    def __init__(self, strategy, num_actions, device):
        self.current_step = 0
        self.strategy = strategy
        self.num_actions = num_actions
        self.device = device

    def select_action(self, state, policy_net):
        rate = strategy.get_exploration_rate(self.current_step)
        self.current_step += 1

        if rate > random.random():
            action = random.randrange(self.num_actions)
            return torch.tensor([action]).to(self.device) # explore      
        else:
            with torch.no_grad():
                return policy_net(state).argmax(dim=1).to(self.device) # exploit

Import Library...

Setup Dsiplay...

Define class DQN...

Create Class Experience for replay memory...

creat instance e ...
Experience(state=2, action=3, next_state=1, reward=4)

Define ReplayMemory()...

define class EpsilonGreedyStrategy()...

Define class Agent()...


# 16.8 Complete Code

In [17]:
# Import Libraries
print('Import Library...')
print()
%matplotlib inline
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

## https://github.com/pytorch/vision
## conda install torchvision -c pytorch

# https://stackoverflow.com/questions/59621736/despite-installing-the-torch-vision-pytorch-library-i-am-getting-an-error-sayin
# conda install pytorch torchvision cpuonly -c pytorch
import torchvision.transforms as T

# Setup Display for plotting images
print('Setup Dsiplay...')
print()
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython: from IPython import display
    
# Define Class DQN and inherit torch.nn.Module 
print('Define class DQN...')
print()
class DQN(nn.Module):
    def __init__(self, img_height, img_width):
        super().__init__()
         
        self.fc1 = nn.Linear(in_features=img_height*img_width*3, out_features=24)   
        self.fc2 = nn.Linear(in_features=24, out_features=32)
        self.out = nn.Linear(in_features=32, out_features=2)            

    def forward(self, t):
        t = t.flatten(start_dim=1)
        t = F.relu(self.fc1(t))
        t = F.relu(self.fc2(t))
        t = self.out(t)
        return t

# Use factory function 'namedtuple' to craete the class 'Experience'.
#
# How to use factory fuction 'namedtuple' to create the class?
# https://realpython.com/python-namedtuple/
#
print('Create Class Experience for replay memory...')
print()
Experience = namedtuple(
    'Experience',
    ('state', 'action', 'next_state', 'reward')
)

# Create expeirnece instance
print('creat instance e ...')
print(e)
print()
e = Experience(2,3,1,4)

print('Define ReplayMemory()...')
print()
class ReplayMemory():
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.push_count = 0
        
    def push(self, experience):
        if len(self.memory) < self.capacity:
            self.memory.append(experience)
        else:
            self.memory[self.push_count % self.capacity] = experience
        self.push_count += 1

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def can_provide_sample(self, batch_size):
        return len(self.memory) >= batch_size

print('define class EpsilonGreedyStrategy()...')
print()
class EpsilonGreedyStrategy():
    def __init__(self, start, end, decay):
        self.start = start
        self.end = end
        self.decay = decay
    
    def get_exploration_rate(self, current_step):
        return self.end + (self.start - self.end) * \
            math.exp(-1. * current_step * self.decay)

    print('Define class Agent()...')
class Agent():
    def __init__(self, strategy, num_actions, device):
        self.current_step = 0
        self.strategy = strategy
        self.num_actions = num_actions
        self.device = device

    def select_action(self, state, policy_net):
        rate = strategy.get_exploration_rate(self.current_step)
        self.current_step += 1

        if rate > random.random():
            action = random.randrange(self.num_actions)
            return torch.tensor([action]).to(self.device) # explore      
        else:
            with torch.no_grad():
                return policy_net(state).argmax(dim=1).to(self.device) # exploit

Import Library...

Setup Dsiplay...

Define class DQN...

Create Class Experience for replay memory...

creat instance e ...
Experience(state=2, action=3, next_state=1, reward=4)

Define ReplayMemory()...

define class EpsilonGreedyStrategy()...

Define class Agent()...
